In [1]:
import tensorflow as tf

2025-09-16 22:02:09.217855: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
shakespeare_url = "https://homl.info/shakespeare"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath, 'r') as f:
    shakespeare_text = f.read()

In [3]:
text_vec_layer = tf.keras.layers.TextVectorization(split='character', standardize='lower')
text_vec_layer.adapt([shakespeare_text])

encoded = text_vec_layer([[shakespeare_text]])[0]
encoded -= 2 #drop [PAD] and [UNK] 每个向前移动两个index
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

I0000 00:00:1758027734.492302  109272 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 166 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5
I0000 00:00:1758027734.526831  109272 cuda_executor.cc:508] failed to allocate 166.31MiB (174391296 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [4]:
text_vec_layer.get_vocabulary()[:4]

['', '[UNK]', np.str_(' '), np.str_('e')]

In [ ]:
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence) # => dataset
    ds = ds.window(length + 1, shift=1, drop_remainder=True) # => dataset of dataset
    ds = ds.flat_map(lambda window_ds:window_ds.batch(length + 1)) # => dataset of tensor
    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size) #再打包

    """
    .flat_map 将dataset转换成张量在拼接起来 #dataset of dataset => dataset of tensor
    .batch 将数据按照batch_size打包成张量
    ds.flat_map(lambda x:x.batch()) 一般跟在.window后面 batch打包window dataset 成tensor。 .flat_map把tensor拼接成dataset
    """

In [35]:
ds = tf.data.Dataset.from_tensor_slices(encoded)
ds_w = ds.window(5, shift=1,drop_remainder=True)
ds_f = ds_w.flat_map(lambda window_ds: window_ds.batch(5))
ds_b = ds_f.batch(3)
for i in ds_b.take(3):
    print(i)

tf.Tensor(
[[19  5  8  7  2]
 [ 5  8  7  2  0]
 [ 8  7  2  0 18]], shape=(3, 5), dtype=int64)
tf.Tensor(
[[ 7  2  0 18  5]
 [ 2  0 18  5  2]
 [ 0 18  5  2  5]], shape=(3, 5), dtype=int64)
tf.Tensor(
[[18  5  2  5 35]
 [ 5  2  5 35  1]
 [ 2  5 35  1  9]], shape=(3, 5), dtype=int64)


In [37]:
for i in ds_b.map(lambda window:(window[:,:-1],window[:,1:])).take(3):
    print(i)

(<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[19,  5,  8,  7],
       [ 5,  8,  7,  2],
       [ 8,  7,  2,  0]])>, <tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 5,  8,  7,  2],
       [ 8,  7,  2,  0],
       [ 7,  2,  0, 18]])>)
(<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 7,  2,  0, 18],
       [ 2,  0, 18,  5],
       [ 0, 18,  5,  2]])>, <tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 2,  0, 18,  5],
       [ 0, 18,  5,  2],
       [18,  5,  2,  5]])>)
(<tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[18,  5,  2,  5],
       [ 5,  2,  5, 35],
       [ 2,  5, 35,  1]])>, <tf.Tensor: shape=(3, 4), dtype=int64, numpy=
array([[ 5,  2,  5, 35],
       [ 2,  5, 35,  1],
       [ 5, 35,  1,  9]])>)


In [27]:
for i in ds_w.flat_map(lambda x:x).take(3):
    print(i)

tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)


In [23]:
encoded.numpy()[:10]

array([19,  5,  8,  7,  2,  0, 18,  5,  2,  5])

In [19]:
for i in ds.take(3):
    print(i)

tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)


In [20]:
for i in ds_w.take(3):
    for j in i.batch(5):
        print(j)

tf.Tensor([19  5  8  7  2], shape=(5,), dtype=int64)
tf.Tensor([5 8 7 2 0], shape=(5,), dtype=int64)
tf.Tensor([ 8  7  2  0 18], shape=(5,), dtype=int64)


2025-09-16 22:16:09.578603: W tensorflow/core/framework/dataset.cc:993] Input of Window will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


In [22]:
for i in ds_f.take(3):
    print(i)

tf.Tensor([19  5  8  7  2], shape=(5,), dtype=int64)
tf.Tensor([5 8 7 2 0], shape=(5,), dtype=int64)
tf.Tensor([ 8  7  2  0 18], shape=(5,), dtype=int64)


In [9]:
for i in ds.take(3):
    print(i)

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


2025-09-16 22:05:36.906325: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [24]:
for i in ds.take(2):
    print(i)

<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>
<_VariantDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>


2025-09-16 19:21:32.321263: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
